<a href="https://colab.research.google.com/github/AndrewwBC/treinamento-h2ia/blob/main/Buscas_sem_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [34]:
import numpy as np
from typing import Literal
import time

class Data_Structure:
  def __init__(self):
    self.items = []

  def is_empty(self):
    return self.items == []

  def push(self, item):
    self.items.append(item)

  def pop_as_a_stack(self):
    return self.items.pop()

  def pop_as_a_queue(self):
    return self.items.pop(0)

  def __str__(self):
    return str(self.items)

class Problem:
    def __init__(self, initial_state):
        self.initial_state = initial_state

class Node:
    def __init__(self, state, parent=None, action=None, depth=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth

    def __str__(self):
        return str(self.state)

    def get_depth(self):
      return self.depth

a = np.arange(0,9)
puzzle = np.array([
    [1, 3, 0],
    [4, 2, 6],
    [7, 5, 8]
])

problem = Problem(puzzle.copy())

target_puzzle = puzzle = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
])

def find_zero(puzzle):
  loc = np.where(puzzle == 0)
  return loc[0][0], loc[1][0]

def actions_by_neighbors(puzzle):
  row, col = find_zero(puzzle)
  neighbors = []

  if row > 0:
    neighbors.append("up")
  if row < 2:
    neighbors.append("down")
  if col > 0:
    neighbors.append("left")
  if col < 2:
    neighbors.append("right")
  return neighbors

def move_zero_to(state, action):
  row, col = find_zero(state)
  aux = state.copy()

  if action == "up":
    aux[row][col] = aux[row - 1][col]
    aux[row - 1][col] = 0

  if action == "down":
    aux[row][col] = aux[row + 1][col]
    aux[row + 1][col] = 0

  if action == "right":
    aux[row][col] = aux[row][col + 1]
    aux[row][col + 1] = 0

  if action == "left":
    aux[row][col] = aux[row][col - 1]
    aux[row][col - 1] = 0

  return aux

def expand(node, state):
  successors = {}

  for action in actions_by_neighbors(state):

    result_of_action = move_zero_to(state, action)

    child = Node(state=result_of_action, parent=node, action=action, depth=node.depth + 1)
    successors[action] = child
  return successors


def freeze(state):
    return tuple(map(tuple, state))

SearchType = Literal["DFS", "BFS"]

def search(option: SearchType):
  fringe = Data_Structure()
  fringe.push(Node(state = problem.initial_state))
  max_depth = -1
  while True:
    if (fringe.is_empty()):
      visited.clear()
      return {"node": node, "message": "Fail", "depth": max_depth}

    if option == "DFS":
      node = fringe.pop_as_a_stack()
    else:
      node = fringe.pop_as_a_queue()

    state_key = freeze(node.state)
    if node.depth > max_depth:
      max_depth = node.depth

    if state_key in visited:
        continue
    visited.add(state_key)

    if np.array_equal(node.state, target_puzzle):
      visited.clear()
      return {"node": node, "message": "Success", "depth": max_depth}

    else:
      for child in expand(node, node.state).values():
        child_key = freeze(child.state)
        if child_key not in visited:
          fringe.push(child)

def IDDFS(limit):
  fringe = Data_Structure()
  fringe.push(Node(state = problem.initial_state))
  max_depth = -1
  while True:
    if (fringe.is_empty()):
      visited.clear()
      return {"message": "Fail", "depth": max_depth}

    node = fringe.pop_as_a_stack()
    state_key = freeze(node.state)

    if node.depth > max_depth:
      max_depth = node.depth

    if state_key in visited:
        continue

    visited.add(state_key)

    if np.array_equal(node.state, target_puzzle):
      visited.clear()
      return {"node": node, "message": "Success", "depth": max_depth}

    else:
      if node.depth < limit:
        for child in expand(node, node.state).values():
          child_key = freeze(child.state)
          if child_key not in visited:
            fringe.push(child)

visited = set()
def call_and_temporize(type_of_search):
  search_begin = time.time()

  if type_of_search == "ID-DFS":
    limit = 0
    max_depth = 50
    while limit < max_depth:
      limit = limit + 1
      result = IDDFS(limit)
      if result["message"] != 'Fail':
        break
  else:
    result = search(type_of_search)

  search_end = time.time()

  return {
      "time": round(search_end - search_begin, 5),
      "depth": result.get("depth")
      }

DFS = call_and_temporize("DFS")
BFS = call_and_temporize("BFS")
IDDFS = call_and_temporize("ID-DFS")

print("\n")
print(100*"-")
print("Tempo (Segundos) -- Profundidade \n")
print("DFS: ", DFS["time"], "-- ", DFS["depth"])
print("BFS: ", BFS["time"], "-- ", BFS["depth"])
print("ID-DFS: ", IDDFS["time"], "-- ", IDDFS["depth"])



----------------------------------------------------------------------------------------------------
Tempo (Segundos) -- Profundidade 

DFS:  1.91146 --  47664
BFS:  0.00091 --  4
ID-DFS:  0.00081 --  4


## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento


 1 - DFS

 A Busca em profundidade possui o menor consumo de memória, pois a memória necessária está atrelada a profundidade da árvore e as ramificações de um ramo apenas. Então, o número médio de ações de cada nó multiplicado pela profundidade gera a complexidade de memória O(número médio de ações * profundidade máxima).

 2 - BFS

Em contrapartida, BFS irá enfileirar todas as ações possíveis a partir de cada nó. Então se cada nó possuir uma média de duas ações, significa que o nó inicial terá dois filhos, quatro netos, oito bisnetos, etc. Ou seja, a complexidade de memória é exponencial: O(número médio de ações^profundidade da solução).

3 - IDDFS

Esta estratégia de busca visa combinar as vantagens de ambos algoritmos anteriormente citados. Apesar de usar um "DFS", ele explora com profundidade crescente, ou seja, pode encontrar a solução mais rapidamente sem explorar todo espaço de possibilidades desnecessariamente, ao mesmo tempo que não consome toda memória que o BFS requer. A complexidade de memória é a mesma do DFS, por óbvio.

